In [175]:
from __future__ import print_function
from functools import reduce
import json
import os
import re
import tarfile
import tempfile

import numpy as np

import keras
import keras.backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import merge, recurrent, Dense, Input, Dropout, TimeDistributed
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.layers.wrappers import Bidirectional
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.regularizers import l2
from keras.utils import np_utils
from numpy import argmax
from sklearn.metrics import confusion_matrix
import pandas as pd

np.random.seed(56) 

In [157]:
def extract_tokens_from_binary_parse(parse):
    return parse.replace('(', ' ').replace(')', ' ').replace('-LRB-', '(').replace('-RRB-', ')').split()

def yield_examples(fn, skip_no_majority=True, limit=None):
    for i, line in enumerate(open(fn)):
        if limit and i > limit:
            break
        data = json.loads(line)
        label = data['gold_label']
        s1 = ' '.join(extract_tokens_from_binary_parse(data['sentence1_binary_parse']))
        s2 = ' '.join(extract_tokens_from_binary_parse(data['sentence2_binary_parse']))
        if skip_no_majority and label == '-':
            continue
        yield (label, s1, s2)

def get_data(fn, limit=None):
    raw_data = list(yield_examples(fn=fn, limit=limit))
    left = [s1 for _, s1, s2 in raw_data]
    right = [s2 for _, s1, s2 in raw_data]
    print(max(len(x.split()) for x in left))
    print(max(len(x.split()) for x in right))

    LABELS = {'contradiction': 0, 'neutral': 1, 'entailment': 2}
    Y = np.array([LABELS[l] for l, s1, s2 in raw_data])
    Y = np_utils.to_categorical(Y, len(LABELS))

    return left, right, Y

def predict_inference(model,text,hypothese):
    prepare_data_pred = lambda data: (to_seq(data[0]), to_seq(data[1]))
    coded=prepare_data_pred([[text],[hypothese]])
    results= model.predict([coded[0],coded[1]])

    inference=LABELS_INV[argmax(results)]
    print("Precicted Judgments: ",inference)
    return inference

training_org = get_data('snli_1.0_train.jsonl')
validation_org = get_data('snli_1.0_dev.jsonl')
test_org = get_data('snli_1.0_test.jsonl')

tokenizer = Tokenizer(lower=False, filters='')
tokenizer.fit_on_texts(training_org[0] + training_org[1])

VOCAB = len(tokenizer.word_counts) + 1
LABELS = {'contradiction': 0, 'neutral': 1, 'entailment': 2}
LABELS_INV = dict((v,k) for k,v in LABELS.items())
RNN = None
LAYERS = 1
USE_GLOVE = True
TRAIN_EMBED = False
EMBED_HIDDEN_SIZE = 300
SENT_HIDDEN_SIZE = 300
BATCH_SIZE = 256
PATIENCE = 4 # 8
MAX_EPOCHS = 10
MAX_LEN = 42
DP = 0.2
L2 = 4e-6
ACTIVATION = 'relu'
OPTIMIZER = 'rmsprop'
print('RNN / Embed / Sent = {}, {}, {}'.format(RNN, EMBED_HIDDEN_SIZE, SENT_HIDDEN_SIZE))
print('GloVe / Trainable Word Embeddings = {}, {}'.format(USE_GLOVE, TRAIN_EMBED))

to_seq = lambda X: pad_sequences(tokenizer.texts_to_sequences(X), maxlen=MAX_LEN)
prepare_data = lambda data: (to_seq(data[0]), to_seq(data[1]), data[2])
prepare_data_pred = lambda data: (to_seq(data[0]), to_seq(data[1]))

training = prepare_data(training_org)
validation = prepare_data(validation_org)
test = prepare_data(test_org)

print('Build model...')
print('Vocab size =', VOCAB)

GLOVE_STORE = 'precomputed_glove.weights'
if USE_GLOVE:
    if not os.path.exists(GLOVE_STORE + '.npy'):
        print('Computing GloVe')
        embeddings_index = {}
        f = open('glove.840B.300d.txt')
        for line in f:
            values = line.split(' ')
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
        f.close()

        embedding_matrix = np.zeros((VOCAB, EMBED_HIDDEN_SIZE))
        for word, i in tokenizer.word_index.items():
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector
            else:
                print('Missing from GloVe: {}'.format(word))

        np.save(GLOVE_STORE, embedding_matrix)

    print('Loading GloVe')
    embedding_matrix = np.load(GLOVE_STORE + '.npy')

    print('Total number of null word embeddings:')
    print(np.sum(np.sum(embedding_matrix, axis=1) == 0))

    embed = Embedding(VOCAB, EMBED_HIDDEN_SIZE, weights=[embedding_matrix], input_length=MAX_LEN, trainable=TRAIN_EMBED)
else:
    embed = Embedding(VOCAB, EMBED_HIDDEN_SIZE, input_length=MAX_LEN)

rnn_kwargs = dict(output_dim=SENT_HIDDEN_SIZE, dropout_W=DP, dropout_U=DP)
SumEmbeddings = keras.layers.core.Lambda(lambda x: K.sum(x, axis=1), output_shape=(SENT_HIDDEN_SIZE, ))

translate = TimeDistributed(Dense(SENT_HIDDEN_SIZE, activation=ACTIVATION))

premise = Input(shape=(MAX_LEN,), dtype='int32')
hypothesis = Input(shape=(MAX_LEN,), dtype='int32')

prem = embed(premise)
hypo = embed(hypothesis)

prem = translate(prem)
hypo = translate(hypo)

if RNN and LAYERS > 1:
    for l in range(LAYERS - 1):
        rnn = RNN(return_sequences=True, **rnn_kwargs)
        prem = BatchNormalization()(rnn(prem))
        hypo = BatchNormalization()(rnn(hypo))
rnn = SumEmbeddings if not RNN else RNN(return_sequences=False, **rnn_kwargs)
prem = rnn(prem)
hypo = rnn(hypo)
prem = BatchNormalization()(prem)
hypo = BatchNormalization()(hypo)

82
62
59
55
57
30
RNN / Embed / Sent = None, 300, 300
GloVe / Trainable Word Embeddings = True, False
Build model...
Vocab size = 42391
Loading GloVe
Total number of null word embeddings:
4043


In [158]:
joint = merge.concatenate([prem, hypo])
joint = Dropout(DP)(joint)
for i in range(3):
    joint = Dense(2 * SENT_HIDDEN_SIZE, activation=ACTIVATION, W_regularizer=l2(L2) if L2 else None)(joint)
    joint = Dropout(DP)(joint)
    joint = BatchNormalization()(joint)

pred = Dense(len(LABELS), activation='softmax')(joint)

model = Model(input=[premise, hypothesis], output=pred)
model.compile(optimizer=OPTIMIZER, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

print('Training')
_, tmpfn = tempfile.mkstemp()

callbacks = [EarlyStopping(patience=PATIENCE), ModelCheckpoint(tmpfn, save_best_only=True, save_weights_only=True)]
model.fit([training[0], training[1]], training[2], batch_size=BATCH_SIZE, nb_epoch=MAX_EPOCHS, validation_data=([validation[0], validation[1]], validation[2]), callbacks=callbacks)

model.load_weights(tmpfn)

loss, acc = model.evaluate([test[0], test[1]], test[2], batch_size=BATCH_SIZE)

print('Test loss / test accuracy = {:.4f} / {:.4f}'.format(loss, acc))


/Users/lovelife/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(600, activation="relu", kernel_regularizer=<keras.reg...)`
  after removing the cwd from sys.path.
/Users/lovelife/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`
  # Remove the CWD from sys.path while we load stuff.
/Users/lovelife/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 42)           0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           (None, 42)           0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 42, 300)      12717300    input_9[0][0]                    
                                                                 input_10[0][0]                   
__________________________________________________________________________________________________
time_distributed_5 (TimeDistrib (None, 42, 300)      90300       embedding_5[0][0]                
          

In [159]:
for i in range(6,17):
    text=test_org[0][i-1]
    hypothese=test_org[1][i-1]
    print("***"*30)
    print(i)
    print("Text: ",text)
    print("Hypothesis: ",hypothese)

******************************************************************************************
6
Text:  A woman with a green headscarf , blue shirt and a very big grin .
Hypothesis:  The woman has been shot .
******************************************************************************************
7
Text:  An old man with a package poses in front of an advertisement .
Hypothesis:  A man poses in front of an ad .
******************************************************************************************
8
Text:  An old man with a package poses in front of an advertisement .
Hypothesis:  A man poses in front of an ad for beer .
******************************************************************************************
9
Text:  An old man with a package poses in front of an advertisement .
Hypothesis:  A man walks by an ad .
******************************************************************************************
10
Text:  A statue at a museum that no seems to be looking at .
Hypothesis:  The

In [160]:
i=1
while i<=11:
    print("     ")
    print("Round "+str(i)+" : please input")
    text = input("Text: ")
    hypothese=input("Hypothesis: ")
    inferenc=predict_inference(model,text,hypothese)
    i+=1

     
Round 1 : please input
Text: A woman with a green headscarf , blue shirt and a very big grin .
Hypothesis: The woman has been shot .
Precicted Judgments:  entailment
     
Round 2 : please input
Text: An old man with a package poses in front of an advertisement .
Hypothesis: A man poses in front of an ad .
Precicted Judgments:  entailment
     
Round 3 : please input
Text: An old man with a package poses in front of an advertisement .
Hypothesis: A man poses in front of an ad for beer .
Precicted Judgments:  neutral
     
Round 4 : please input
Text: An old man with a package poses in front of an advertisement .
Hypothesis: A man walks by an ad .
Precicted Judgments:  contradiction
     
Round 5 : please input
Text: A statue at a museum that no seems to be looking at .
Hypothesis: The statue is offensive and people are mad that it is on display .
Precicted Judgments:  neutral
     
Round 6 : please input
Text: A statue at a museum that no seems to be looking at .
Hypothesis: Ther

KeyboardInterrupt: 

In [173]:
y_train_predicted= model.predict([training[0], training[1]])
y_train_predicted=[argmax(x) for x in y_train_predicted]
y_train_real=[argmax(x) for x in training[2]]

y_valid_predicted= model.predict([validation[0], validation[1]])
y_valid_predicted=[argmax(x) for x in y_valid_predicted]
y_valid_real=[argmax(x) for x in validation[2]]

y_test_predicted= model.predict([test[0], test[1]])
y_test_predicted=[argmax(x) for x in y_test_predicted]
y_test_real=[argmax(x) for x in test[2]]
y_predicted
y_real

[1,
 2,
 0,
 1,
 2,
 0,
 2,
 1,
 0,
 1,
 2,
 0,
 2,
 2,
 0,
 0,
 2,
 1,
 2,
 0,
 1,
 0,
 2,
 1,
 1,
 0,
 2,
 2,
 1,
 0,
 0,
 2,
 2,
 2,
 1,
 0,
 1,
 2,
 0,
 2,
 1,
 0,
 0,
 2,
 1,
 1,
 0,
 2,
 1,
 0,
 1,
 2,
 0,
 1,
 0,
 2,
 2,
 0,
 1,
 2,
 0,
 1,
 1,
 2,
 0,
 2,
 1,
 1,
 0,
 1,
 2,
 0,
 2,
 1,
 0,
 2,
 1,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 0,
 1,
 0,
 1,
 2,
 1,
 0,
 2,
 1,
 2,
 0,
 0,
 2,
 1,
 1,
 2,
 0,
 1,
 2,
 0,
 2,
 1,
 2,
 0,
 2,
 2,
 2,
 0,
 1,
 2,
 0,
 1,
 2,
 0,
 0,
 2,
 1,
 1,
 0,
 1,
 2,
 0,
 1,
 2,
 0,
 0,
 1,
 2,
 1,
 2,
 0,
 2,
 0,
 1,
 2,
 0,
 1,
 1,
 2,
 0,
 1,
 0,
 2,
 1,
 1,
 0,
 2,
 1,
 0,
 0,
 1,
 2,
 1,
 0,
 2,
 1,
 2,
 0,
 1,
 1,
 0,
 2,
 0,
 1,
 0,
 2,
 2,
 2,
 0,
 1,
 2,
 0,
 1,
 1,
 0,
 2,
 1,
 2,
 1,
 1,
 2,
 0,
 0,
 1,
 2,
 2,
 1,
 0,
 2,
 1,
 0,
 2,
 0,
 1,
 0,
 1,
 2,
 1,
 0,
 1,
 2,
 0,
 2,
 1,
 0,
 0,
 2,
 1,
 0,
 2,
 1,
 2,
 1,
 0,
 2,
 0,
 1,
 1,
 2,
 0,
 2,
 2,
 0,
 0,
 2,
 2,
 1,
 0,
 1,
 1,
 2,
 0,
 2,
 0,
 1,
 0,
 1,
 2,
 2,
 2,
 0,
 2,
 1,
 0,
 1,
 0,


In [176]:
confusion_columns=['pred_contradiction','pred_neutral','pred_entailment']
confusion_rows=['true_contradiction','true_neutral','true_entailment']
train_confusion_matrix=pd.DataFrame(confusion_matrix(y_train_real, y_train_predicted))
train_confusion_matrix.columns=confusion_columns
train_confusion_matrix['index']=confusion_rows
train_confusion_matrix=train_confusion_matrix.set_index('index')
train_confusion_matrix.index.name=''

valid_confusion_matrix=pd.DataFrame(confusion_matrix(y_valid_real, y_valid_predicted))
valid_confusion_matrix.columns=confusion_columns
valid_confusion_matrix['index']=confusion_rows 
valid_confusion_matrix=valid_confusion_matrix.set_index('index')
valid_confusion_matrix.index.name=''

test_confusion_matrix=pd.DataFrame(confusion_matrix(y_test_real, y_test_predicted))
test_confusion_matrix.columns=confusion_columns
test_confusion_matrix['index']=confusion_rows
test_confusion_matrix=test_confusion_matrix.set_index('index')
test_confusion_matrix.index.name=''

display(train_confusion_matrix)
display(valid_confusion_matrix)
display(test_confusion_matrix)


,pred_contradiction,pred_neutral,pred_entailment
,,,
true_contradiction,147628,20826,14733
true_neutral,9921,141736,31107
true_entailment,3161,12554,167701


,pred_contradiction,pred_neutral,pred_entailment
,,,
true_contradiction,2582,387,309
true_neutral,235,2495,505
true_entailment,81,224,3024


,pred_contradiction,pred_neutral,pred_entailment
,,,
true_contradiction,2524,404,309
true_neutral,231,2414,574
true_entailment,89,257,3022
